# Chapter 8 - 객체 참조, 가변성, 재활용

## 변수는 상자가 아니다

파이썬 변수는 자바에서의 참조 변수와 같으므로 변수는 객체에 붙은 레이블이라고 생각하는 것이 좋다.

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
print(b)

[1, 2, 3, 4]


## 정체성, 동질성, 별명

In [2]:
a = {'qwer': 1, 'asdf': 2}
b = a
print(a is b)

True


In [3]:
c = {'qwer': 1, 'asdf': 2}
print(a == c)
print(a is c)

True
False


In [4]:
b['zxcv'] = 3
print(a)

{'qwer': 1, 'asdf': 2, 'zxcv': 3}


In [5]:
print(a == c)
print(a is c)

False
False


객체 id의 실제 의미는 구현에 따라 다르다. CPython의 경우 id()는 객체의 메모리 주소를 반환하지만,<br>
다른 파이썬 인터프리터는 메모리 주소 이외의 다른 값을 반환할 수도 있다.<br>
실제로 프로그래밍할 때는 id() 함수를 거의 사용하지 않는다.

## 기본 복사는 얕은 복사

In [6]:
a = [100, [11, 22, 33], (-1, -2, -3)]
b = a[:]

a.append(50)
print(a)
print(b)

[100, [11, 22, 33], (-1, -2, -3), 50]
[100, [11, 22, 33], (-1, -2, -3)]


In [7]:
a[1] += [44, 55]
print(a)
print(b)

[100, [11, 22, 33, 44, 55], (-1, -2, -3), 50]
[100, [11, 22, 33, 44, 55], (-1, -2, -3)]


In [8]:
b[2] += (-4, -5)
print(a)
print(b)

[100, [11, 22, 33, 44, 55], (-1, -2, -3), 50]
[100, [11, 22, 33, 44, 55], (-1, -2, -3, -4, -5)]


튜플은 불변형이기 때문에 ```b[2] += (-4, -5)```를 하면 새로운 객체를 만들어서 바인딩한다.<br>
깊은 복사를 하고 싶을 땐 copy 모듈의 deepcopy() 함수를 사용하면 된다.

## 참조로서의 함수 매개변수

In [9]:
def f(a, b):
    a += b
    return a

x, y = 1, 2
print(f(x, y))
print(x, y)

a, b = [1, 2], [3, 4]
print(f(a, b))
print(a, b)

p, q = (10, 20), (30, 40)
print(f(p, q))
print(p, q)

3
1 2
[1, 2, 3, 4]
[1, 2, 3, 4] [3, 4]
(10, 20, 30, 40)
(10, 20) (30, 40)


## del과 가비지 컬렉션

del 명령은 이름을 제거하는 것이지, 객체를 제거하는 것이 아니다.<br>
CPython의 경우 가비지 컬렉션은 주로 참조 카운트에 기반하며, 이 카운트가 0이 되자마자<br>
CPython이 객체의 \_\_del\_\_() 메서드를 호출하고(정의 되어있는 경우) 객체에 할당되어 있는 메모리를 해제함으로써 객체가 제거된다.

In [1]:
import weakref

s1 = {1, 2, 3}
s2 = s1
def bye():
    print('Good Bye')
    
ender = weakref.finalize(s1, bye)
print(ender.alive)

del s1
print(ender.alive)

s2 = 'asdf'
print(ender.alive)

True
True
Good Bye
False


## 파이썬의 특이한 불변형 처리법

튜플 t에 대해 t[:]나 tuple(t)는 사본을 생성하지 않고 그 객체에 대한 참조를 반환한다.<br>
또한 문자열 리터럴을 공유하는 최적화 기법을 **인터닝(interning)** 이라고 한다.<br>
Cpython에서는 0, -1, 42 등 작은 정수에도 동일한 기법을 적용하지만 모든 문자열이나 정수를 인터닝하는 것은 아니며<br>
절대로 str이나 int형의 인터닝에 의존하면 안 된다!

In [1]:
a = 'asdf'
b = 'asdf'

print(a is b)

True
